<a href="https://colab.research.google.com/github/nasifnur307-blip/biomarkerlink/blob/main/notebooks/setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install numpy scipy matplotlib scikit-learn pandas networkx pgmpy causal-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.0/193.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 19.0 MB/s eta 0:00:00


In [2]:
!pip install causal-learn==0.1.4.3



In [3]:
# lowercase causallearn
from causallearn.search.ConstraintBased.PC import pc


In [4]:
from sklearn.preprocessing import StandardScaler


In [5]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from causallearn.search.ConstraintBased.PC import pc

print("Ready.")


Ready.


In [6]:
from google.colab import drive
drive.mount('/content/drive')

!git clone https://github.com/nasifnur307-blip/biomarkerlink.git


Mounted at /content/drive
Cloning into 'biomarkerlink'...
remote: Enumerating objects: 41, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 41 (delta 11), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (41/41), 44.43 KiB | 3.42 MiB/s, done.
Resolving deltas: 100% (11/11), done.


In [7]:
import os

folders = [
    "data/raw",
    "data/processed",
    "results/graphs",
    "results/models",
]

for f in folders:
    os.makedirs(f, exist_ok=True)

print("Folders created.")


Folders created.


In [8]:
df = pd.read_csv('/content/drive/MyDrive/NeuroFedMeta/data/CHBMIT/chbmit_preprocessed_data.csv')

print("Dataset shape:", df.shape)
print("Columns:", df.columns.tolist())
print("Label distribution:\n", df['Outcome'].value_counts())

Dataset shape: (2097150, 24)
Columns: ['# FP1-F7', 'C3-P3', 'C4-P4', 'CZ-PZ', 'F3-C3', 'F4-C4', 'F7-T7', 'F8-T8', 'FP1-F3', 'FP2-F4', 'FP2-F8', 'FT10-T8', 'FT9-FT10', 'FZ-CZ', 'P3-O1', 'P4-O2', 'P7-O1', 'P7-T7', 'P8-O2', 'T7-FT9', 'T7-P7', 'T8-P8-0', 'T8-P8-1', 'Outcome']
Label distribution:
 Outcome
0.0    1048575
1.0    1048575
Name: count, dtype: int64


In [9]:
# Features = EEG channels (first 23 columns)
X = df.iloc[:, :-1].values
# Outcome column = inter-/pre-/ictal labels
y = df['Outcome'].values

print("Features shape:", X.shape)
print("Labels shape:", y.shape)


Features shape: (2097150, 23)
Labels shape: (2097150,)


In [10]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [11]:
# Inter-ictal: Outcome = 0
X_inter = X_scaled[y == 0]

# Pre-ictal: Outcome = 1 (adjust if you have explicit pre-ictal vs ictal labels)
X_pre = X_scaled[y == 1]

print("Inter-ictal shape:", X_inter.shape)
print("Pre-ictal shape:", X_pre.shape)


Inter-ictal shape: (1048575, 23)
Pre-ictal shape: (1048575, 23)


In [12]:

alpha = 0.05


is_identical = df[df.columns[-3]].equals(df[df.columns[-2]])
print(f"Are columns '{df.columns[-3]}' and '{df.columns[-2]}' identical? {is_identical}")

if is_identical:
    print(f"Dropping column '{df.columns[-2]}' due to duplication.")
    # Re-extract features, excluding the duplicate column
    features_to_use = [col for col in df.columns[:-1] if col != df.columns[-2]]
    X_fixed = df[features_to_use].values


    scaler_fixed = StandardScaler()
    X_scaled_fixed = scaler_fixed.fit_transform(X_fixed)


    X_inter_fixed = X_scaled_fixed[y == 0]
    X_pre_fixed = X_scaled_fixed[y == 1]

    print("Running PC algorithm with fixed data...")
    causal_graph_inter = pc(data=X_inter_fixed, alpha=alpha, method_ind='hsic')
else:
    print("Columns are not identical, running PC with original data...")
    causal_graph_inter = pc(data=X_inter, alpha=alpha, method_ind='hsic')


print("PC algorithm finished for inter-ictal state.")

Are columns 'T8-P8-0' and 'T8-P8-1' identical? True
Dropping column 'T8-P8-1' due to duplication.
Running PC algorithm with fixed data...


  0%|          | 0/22 [00:00<?, ?it/s]

PC algorithm finished for inter-ictal state.


In [13]:
print(type(causal_graph_inter))
print(dir(causal_graph_inter))

<class 'causallearn.graph.GraphClass.CausalGraph'>
['G', 'PC_elapsed', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slotnames__', '__str__', '__subclasshook__', '__weakref__', 'ci_test', 'definite_UC', 'definite_non_UC', 'draw_nx_graph', 'draw_pydot_graph', 'find_adj', 'find_arrow_heads', 'find_bi_directed', 'find_cond_sets', 'find_cond_sets_with_mid', 'find_cond_sets_without_mid', 'find_fully_directed', 'find_kites', 'find_tails', 'find_triangles', 'find_undirected', 'find_unshielded_triples', 'is_fully_directed', 'is_undirected', 'labels', 'max_degree', 'neighbors', 'nx_graph', 'nx_skel', 'prt_m', 'rearrange', 'redundant_nodes', 'sepset', 'set_ind_test', 'test', 'to_nx_graph', 'to_nx_skeleton']


In [14]:
import networkx as nx

# 1. Access the underlying NetworkX graph representation
inter_graph_nx = causal_graph_inter.to_nx_graph()

# 2. Print the number of nodes and edges in inter_graph_nx
if inter_graph_nx:
    print(f"Inter-ictal Causal Graph (Directed):\n  Number of nodes: {inter_graph_nx.number_of_nodes()}\n  Number of edges: {inter_graph_nx.number_of_edges()}")
else:
    print("Inter-ictal Causal Graph (Directed): No graph generated or graph is empty.")

# 3. Access the underlying NetworkX skeleton (undirected graph) representation
inter_skeleton_nx = causal_graph_inter.to_nx_skeleton()

# 4. Print the number of nodes and edges in inter_skeleton_nx
if inter_skeleton_nx:
    print(f"Inter-ictal Causal Graph Skeleton (Undirected):\n  Number of nodes: {inter_skeleton_nx.number_of_nodes()}\n  Number of edges: {inter_skeleton_nx.number_of_edges()}")
else:
    print("Inter-ictal Causal Graph Skeleton (Undirected): No skeleton generated or skeleton is empty.")

Inter-ictal Causal Graph (Directed): No graph generated or graph is empty.
Inter-ictal Causal Graph Skeleton (Undirected): No skeleton generated or skeleton is empty.


# CHB MIT Preprocess

In [15]:
pip install mne

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 44.1 MB/s eta 0:00:00


In [16]:
import mne
import os

print("MNE-Python and os libraries imported successfully.")

MNE-Python and os libraries imported successfully.


In [37]:
import mne
import os

mne.set_log_level('WARNING')

raw_data_dir = '/content/drive/MyDrive/NeuroFedMeta/data/Raw_CHBMIT/CHB.02_02.edf'
os.makedirs(raw_data_dir, exist_ok=True)


edf_files = [os.path.join(raw_data_dir, f) for f in os.listdir(raw_data_dir) if f.endswith('.edf')]

if not edf_files:
    print(f"No EDF files found in {raw_data_dir}. Please download and place the raw CHBMIT EDF files there.")
else:
    print(f"Found {len(edf_files)} EDF files in {raw_data_dir}.")
    print(f"Total {len(edf_files)} EDF files loaded for processing.")
    print("Example files:", edf_files[:3]) # Print first 3 files if many

# List to store preprocessed MNE Raw objects (or extracted data)
preprocessed_raw_objects = []

# Define preprocessing parameters
low_freq = 0.5
high_freq = 45
new_sfreq = 100 # Hz

if edf_files:
    for i, edf_file in enumerate(edf_files):
        print(f"\nProcessing file {i+1}/{len(edf_files)}: {os.path.basename(edf_file)}")
        try:

            raw = mne.io.read_raw_edf(edf_file, preload=True)
            # Ensure a descriptive name is set for saving later
            raw.info['description'] = os.path.basename(edf_file).replace('.edf', '')
            print(f"  Original sampling frequency: {raw.info['sfreq']} Hz")
            print(f"  Number of channels: {len(raw.ch_names)}")
            print(f"  Duration: {raw.times[-1] / 60:.2f} minutes")

            # Apply common average reference to EEG channels only if they exist
            eeg_channels = mne.pick_types(raw.info, eeg=True, exclude='bads')
            if len(eeg_channels) > 0:
                raw.set_eeg_reference(ref_channels='average')
                print("  Applied Common Average Reference.")
            else:
                print("  No EEG channels found for re-referencing.")

            # 5b. Apply a bandpass filter
            raw.filter(l_freq=low_freq, h_freq=high_freq, picks='eeg')
            print(f"  Applied bandpass filter ({low_freq}-{high_freq} Hz).")

            # 5c. Downsample the Raw object
            raw.resample(sfreq=new_sfreq)
            print(f"  Downsampled to {raw.info['sfreq']} Hz.")

            # 5d. Store the preprocessed Raw object (or its data)
            preprocessed_raw_objects.append(raw) # Store the MNE Raw object

        except Exception as e:
            print(f"  Error processing {os.path.basename(edf_file)}: {e}")

    print(f"\nFinished preprocessing {len(preprocessed_raw_objects)} files.")
else:
    print("Skipping preprocessing as no EDF files were found.")

Found 19 EDF files in /content/drive/MyDrive/NeuroFedMeta/data/Raw_CHBMIT/CHB.02_02.edf.
Total 19 EDF files loaded for processing.
Example files: ['/content/drive/MyDrive/NeuroFedMeta/data/Raw_CHBMIT/CHB.02_02.edf/chb02_13.edf', '/content/drive/MyDrive/NeuroFedMeta/data/Raw_CHBMIT/CHB.02_02.edf/chb02_14.edf', '/content/drive/MyDrive/NeuroFedMeta/data/Raw_CHBMIT/CHB.02_02.edf/chb02_16.edf']

Processing file 1/19: chb02_13.edf


/tmp/ipython-input-3712203312.py:32: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(edf_file, preload=True)


  Original sampling frequency: 256.0 Hz
  Number of channels: 23
  Duration: 60.00 minutes
  Applied Common Average Reference.
  Applied bandpass filter (0.5-45 Hz).
  Downsampled to 100.0 Hz.

Processing file 2/19: chb02_14.edf


/tmp/ipython-input-3712203312.py:32: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(edf_file, preload=True)


  Original sampling frequency: 256.0 Hz
  Number of channels: 23
  Duration: 60.00 minutes
  Applied Common Average Reference.
  Applied bandpass filter (0.5-45 Hz).
  Downsampled to 100.0 Hz.

Processing file 3/19: chb02_16.edf


...

  Original sampling frequency: 256.0 Hz
  Number of channels: 23
  Duration: 60.00 minutes
  Applied Common Average Reference.
  Applied bandpass filter (0.5-45 Hz).
  Downsampled to 100.0 Hz.

Processing file 18/19: chb02_18.edf


/tmp/ipython-input-3712203312.py:32: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(edf_file, preload=True)


  Original sampling frequency: 256.0 Hz
  Number of channels: 23
  Duration: 60.00 minutes
  Applied Common Average Reference.
  Applied bandpass filter (0.5-45 Hz).
  Downsampled to 100.0 Hz.

Processing file 19/19: chb02_19.edf


/tmp/ipython-input-3712203312.py:32: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(edf_file, preload=True)


  Original sampling frequency: 256.0 Hz
  Number of channels: 23
  Duration: 60.00 minutes
  Applied Common Average Reference.
  Applied bandpass filter (0.5-45 Hz).
  Downsampled to 100.0 Hz.

Finished preprocessing 19 files.


In [18]:
import mne
print(f"MNE-Python version: {mne.__version__}")

MNE-Python version: 1.11.0


In [38]:
from mne.preprocessing import ICA


processed_data_dir = '/content/drive/MyDrive/NeuroFedMeta/data/Raw_CHBMIT/processed'
os.makedirs(processed_data_dir, exist_ok=True)


ica_cleaned_fif_paths = []


if preprocessed_raw_objects:
    print(f"\nApplying ICA to {len(preprocessed_raw_objects)} preprocessed Raw objects and saving them...")
    for i, raw_obj in enumerate(preprocessed_raw_objects):
        # Use the description set in the previous step, or a generic name if unavailable
        original_filename = raw_obj.info['description'] if raw_obj.info['description'] else f'unnamed_raw_{i}'
        output_fif_path = os.path.join(processed_data_dir, f'{original_filename}_ica_cleaned.fif')

        # Check if ICA-cleaned file already exists to avoid reprocessing
        if os.path.exists(output_fif_path):
            print(f"  Skipping ICA for {original_filename} as {output_fif_path} already exists.")

            continue

        print(f"  Processing raw object {i+1}/{len(preprocessed_raw_objects)}: {original_filename}")
        try:

            n_components_ica = min(20, len(raw_obj.ch_names))
            ica = ICA(n_components=n_components_ica, random_state=99, verbose=False)


            ica.fit(raw_obj)


            ica_cleaned_raw = ica.apply(raw_obj.copy(), exclude=ica.exclude, verbose=False)
            print(f"    ICA applied. Original channels: {len(raw_obj.ch_names)}, ICA components: {ica.n_components_}, Excluded components: {len(ica.exclude)}")


            ica_cleaned_raw.set_meas_date(None)



            ica_cleaned_raw.save(output_fif_path, overwrite=True, verbose=False)
            print(f"    Saved ICA-cleaned data to: {output_fif_path}")

        except Exception as e:
            print(f"  Error applying ICA to raw object {i+1} ({original_filename}): {e}")


ica_cleaned_fif_paths = [os.path.join(processed_data_dir, f) for f in os.listdir(processed_data_dir) if f.endswith('_ica_cleaned.fif') or f.endswith('_ica_cleaned.fif.gz')]

if ica_cleaned_fif_paths:
    print(f"\nFinished ICA cleaning and saving. Total {len(ica_cleaned_fif_paths)} ICA-cleaned .fif files found on disk.")
    print("List of ICA-cleaned .fif files:", ica_cleaned_fif_paths)
else:
    print("\nNo ICA-cleaned .fif files were found on disk. Please ensure EDF files are present in the raw data directory and processed in previous steps, and ICA runs successfully.")


Applying ICA to 19 preprocessed Raw objects and saving them...
  Processing raw object 1/19: chb02_13
    ICA applied. Original channels: 23, ICA components: 20, Excluded components: 0


/tmp/ipython-input-4269725839.py:42: RuntimeWarning: This filename (/content/drive/MyDrive/NeuroFedMeta/data/Raw_CHBMIT/processed/chb02_13_ica_cleaned.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  ica_cleaned_raw.save(output_fif_path, overwrite=True, verbose=False)


    Saved ICA-cleaned data to: /content/drive/MyDrive/NeuroFedMeta/data/Raw_CHBMIT/processed/chb02_13_ica_cleaned.fif
  Processing raw object 2/19: chb02_14
    ICA applied. Original channels: 23, ICA components: 20, Excluded components: 0


/tmp/ipython-input-4269725839.py:42: RuntimeWarning: This filename (/content/drive/MyDrive/NeuroFedMeta/data/Raw_CHBMIT/processed/chb02_14_ica_cleaned.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  ica_cleaned_raw.save(output_fif_path, overwrite=True, verbose=False)


    Saved ICA-cleaned data to: /content/drive/MyDrive/NeuroFedMeta/data/Raw_CHBMIT/processed/chb02_14_ica_cleaned.fif
  Processing raw object 3/19: chb02_16
    ICA applied. Original channels: 23, ICA components: 20, Excluded components: 0


...

    Saved ICA-cleaned data to: /content/drive/MyDrive/NeuroFedMeta/data/Raw_CHBMIT/processed/chb02_12_ica_cleaned.fif
  Processing raw object 18/19: chb02_18
    ICA applied. Original channels: 23, ICA components: 20, Excluded components: 0


/tmp/ipython-input-4269725839.py:42: RuntimeWarning: This filename (/content/drive/MyDrive/NeuroFedMeta/data/Raw_CHBMIT/processed/chb02_18_ica_cleaned.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  ica_cleaned_raw.save(output_fif_path, overwrite=True, verbose=False)


    Saved ICA-cleaned data to: /content/drive/MyDrive/NeuroFedMeta/data/Raw_CHBMIT/processed/chb02_18_ica_cleaned.fif
  Processing raw object 19/19: chb02_19
    ICA applied. Original channels: 23, ICA components: 20, Excluded components: 0


/tmp/ipython-input-4269725839.py:42: RuntimeWarning: This filename (/content/drive/MyDrive/NeuroFedMeta/data/Raw_CHBMIT/processed/chb02_19_ica_cleaned.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  ica_cleaned_raw.save(output_fif_path, overwrite=True, verbose=False)


    Saved ICA-cleaned data to: /content/drive/MyDrive/NeuroFedMeta/data/Raw_CHBMIT/processed/chb02_19_ica_cleaned.fif

Finished ICA cleaning and saving. Total 61 ICA-cleaned .fif files found on disk.
List of ICA-cleaned .fif files: ['/content/drive/MyDrive/NeuroFedMeta/data/Raw_CHBMIT/processed/chb01_08_ica_cleaned.fif', '/content/drive/MyDrive/NeuroFedMeta/data/Raw_CHBMIT/processed/chb01_01_ica_cleaned.fif', '/content/drive/MyDrive/NeuroFedMeta/data/Raw_CHBMIT/processed/chb01_02_ica_cleaned.fif', '/content/drive/MyDrive/NeuroFedMeta/data/Raw_CHBMIT/processed/chb01_09_ica_cleaned.fif', '/content/drive/MyDrive/NeuroFedMeta/data/Raw_CHBMIT/processed/chb01_03_ica_cleaned.fif', '/content/drive/MyDrive/NeuroFedMeta/data/Raw_CHBMIT/processed/chb01_10_ica_cleaned.fif', '/content/drive/MyDrive/NeuroFedMeta/data/Raw_CHBMIT/processed/chb01_04_ica_cleaned.fif', '/content/drive/MyDrive/NeuroFedMeta/data/Raw_CHBMIT/processed/chb01_05_ica_cleaned.fif', '/content/drive/MyDrive/NeuroFedMeta/data/Raw_C

In [ ]:
import numpy as np

# 1. Define segmentation parameters
segment_duration = 5 # seconds

# 2. Initialize an empty list to store the extracted data from each segment
segmented_data_for_causal_analysis = []

# 3. Iterate through each ICA-cleaned .fif file path
if 'ica_cleaned_fif_paths' in locals() and ica_cleaned_fif_paths:
    print(f"\nSegmenting data from {len(ica_cleaned_fif_paths)} ICA-cleaned .fif files...")
    for i, fif_path in enumerate(ica_cleaned_fif_paths):
        filename = os.path.basename(fif_path)
        print(f"  Segmenting file {i+1}/{len(ica_cleaned_fif_paths)}: {filename}")
        try:
            # Load the ICA-cleaned Raw object from the .fif file
            raw_obj = mne.io.read_raw_fif(fif_path, preload=True, verbose=False)

            # 4. Create non-overlapping epochs (segments) of the defined duration
            epochs = mne.make_fixed_length_epochs(raw_obj, duration=segment_duration, overlap=0, preload=True, verbose=False)
            print(f"    Created {len(epochs)} epochs, each {segment_duration} seconds long.")

            # 5. Extract the numerical data from each epoch and prepare for causal inference
            epochs_data = epochs.get_data(picks='eeg')

            n_epochs, n_channels, n_times = epochs_data.shape
            reshaped_epochs_data = epochs_data.reshape(n_epochs, n_channels * n_times)

            segmented_data_for_causal_analysis.append(reshaped_epochs_data)

            # Explicitly delete objects to free memory after processing a chunk
            del epochs, raw_obj

        except Exception as e:
            print(f"  Error segmenting file {i+1} ({filename}): {e}")

    # Concatenate all segmented data if multiple files were processed
    if segmented_data_for_causal_analysis:
        all_segmented_data = np.vstack(segmented_data_for_causal_analysis)
        print(f"\nFinished segmentation. Processed {len(ica_cleaned_fif_paths)} files and created a total of {all_segmented_data.shape[0]} segments.")
        print(f"Shape of combined segmented data for causal analysis: {all_segmented_data.shape}")
    else:
        print("No segments were successfully created for causal analysis.")
else:
    print("\nNo ICA-cleaned .fif file paths available for segmentation. Please ensure ICA cleaning and saving steps ran successfully.")

In [4]:
import h5py
import os

# 1. Check if segmented data exists
if 'all_segmented_data' in locals() and all_segmented_data.size > 0:
    data_to_save = all_segmented_data
    print(f"Data to save found with shape: {data_to_save.shape}")
elif segmented_data_for_causal_analysis:

    data_to_save = np.vstack(segmented_data_for_causal_analysis)
    print(f"Data to save found in list form with shape: {data_to_save.shape}")
else:
    print("No segmented data available to save. Please ensure previous steps ran successfully.")
    data_to_save = None

if data_to_save is not None:
    # 2. Define filename and path
    processed_data_dir = '/content/drive/MyDrive/NeuroFedMeta/data/Raw_CHBMIT/processed_Final'
    os.makedirs(processed_data_dir, exist_ok=True)
    output_filename = os.path.join(processed_data_dir, 'segmented_eeg_data.h5')

    # 4. Use h5py.File() to create or open an HDF5 file
    with h5py.File(output_filename, 'w') as f:
        # 5. Save the NumPy array as a dataset within the HDF5 file with compression
        f.create_dataset('eeg_segments', data=data_to_save, compression="gzip", compression_opts=9)

    print(f"Successfully saved segmented EEG data to: {output_filename}")
    print(f"Saved data shape: {data_to_save.shape}")


No segmented data available to save. Please ensure previous steps ran successfully.


# Task
Confirm that the segmentation process has successfully completed and that the segmented data is saved correctly by updating the segmentation cell (`LWwfcBTj9PUX`) to explicitly define `processed_data_dir` and re-populate the `ica_cleaned_fif_paths` list by scanning the directory for `.fif` files, then executing the cell to process and save the data to an HDF5 file.

## Modify Segmentation Cell for Robustness

### Subtask:
Update the segmentation cell (LWwfcBTj9PUX) to explicitly define `processed_data_dir` and re-populate the `ica_cleaned_fif_paths` list by scanning the directory for `.fif` files.


**Reasoning**:
The subtask requires modifying cell `LWwfcBTj9PUX` to explicitly define the `processed_data_dir` and then re-populate the `ica_cleaned_fif_paths` list by scanning this directory for `.fif` files. This ensures robustness and explicit handling of file paths.



In [7]:
import numpy as np
import os # Ensure os is imported
import mne # Ensure mne is imported as it's used later in the cell

# Explicitly define processed_data_dir
processed_data_dir = '/content/drive/MyDrive/NeuroFedMeta/data/Raw_CHBMIT/processed'

# Re-populate ica_cleaned_fif_paths by scanning the directory
# This assumes processed_data_dir exists and contains files.
ica_cleaned_fif_paths = [os.path.join(processed_data_dir, f) for f in os.listdir(processed_data_dir) if f.endswith('_ica_cleaned.fif') or f.endswith('_ica_cleaned.fif.gz')]

# 1. Define segmentation parameters
segment_duration = 5 # seconds

# 2. Initialize an empty list to store the extracted data from each segment
segmented_data_for_causal_analysis = []

# 3. Iterate through each ICA-cleaned .fif file path
# The original 'if' condition is simplified since ica_cleaned_fif_paths is now explicitly populated.
if ica_cleaned_fif_paths:
    print(f"\nSegmenting data from {len(ica_cleaned_fif_paths)} ICA-cleaned .fif files...")
    for i, fif_path in enumerate(ica_cleaned_fif_paths):
        filename = os.path.basename(fif_path)
        print(f"  Segmenting file {i+1}/{len(ica_cleaned_fif_paths)}: {filename}")
        try:
            # Load the ICA-cleaned Raw object from the .fif file
            raw_obj = mne.io.read_raw_fif(fif_path, preload=True, verbose=False)

            # 4. Create non-overlapping epochs (segments) of the defined duration
            epochs = mne.make_fixed_length_epochs(raw_obj, duration=segment_duration, overlap=0, preload=True, verbose=False)
            print(f"    Created {len(epochs)} epochs, each {segment_duration} seconds long.")

            # 5. Extract the numerical data from each epoch and prepare for causal inference
            epochs_data = epochs.get_data(picks='eeg')

            n_epochs, n_channels, n_times = epochs_data.shape
            # Reshape each epoch into a single row vector
            # This creates a 2D array where each row is a flattened epoch
            reshaped_epochs_data = epochs_data.reshape(n_epochs, n_channels * n_times)

            segmented_data_for_causal_analysis.append(reshaped_epochs_data)

            # Explicitly delete objects to free memory after processing a chunk
            del epochs, raw_obj

        except Exception as e:
            print(f"  Error segmenting file {i+1} ({filename}): {e}")

    # Concatenate all segmented data if multiple files were processed
    if segmented_data_for_causal_analysis:
        all_segmented_data = np.vstack(segmented_data_for_causal_analysis)
        print(f"\nFinished segmentation. Processed {len(ica_cleaned_fif_paths)} files and created a total of {all_segmented_data.shape[0]} segments.")
        print(f"Shape of combined segmented data for causal analysis: {all_segmented_data.shape}")
    else:
        print("No segments were successfully created for causal analysis.")
else:
    print("\nNo ICA-cleaned .fif file paths found in the specified directory. Please ensure ICA cleaning and saving steps ran successfully.")


Segmenting data from 61 ICA-cleaned .fif files...
  Segmenting file 1/61: chb01_08_ica_cleaned.fif


/tmp/ipython-input-3105503034.py:27: RuntimeWarning: This filename (/content/drive/MyDrive/NeuroFedMeta/data/Raw_CHBMIT/processed/chb01_08_ica_cleaned.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_obj = mne.io.read_raw_fif(fif_path, preload=True, verbose=False)


    Created 720 epochs, each 5 seconds long.
  Segmenting file 2/61: chb01_01_ica_cleaned.fif


/tmp/ipython-input-3105503034.py:27: RuntimeWarning: This filename (/content/drive/MyDrive/NeuroFedMeta/data/Raw_CHBMIT/processed/chb01_01_ica_cleaned.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_obj = mne.io.read_raw_fif(fif_path, preload=True, verbose=False)


    Created 720 epochs, each 5 seconds long.
  Segmenting file 3/61: chb01_02_ica_cleaned.fif


...

    Created 720 epochs, each 5 seconds long.
  Segmenting file 60/61: chb02_18_ica_cleaned.fif


/tmp/ipython-input-3105503034.py:27: RuntimeWarning: This filename (/content/drive/MyDrive/NeuroFedMeta/data/Raw_CHBMIT/processed/chb02_18_ica_cleaned.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_obj = mne.io.read_raw_fif(fif_path, preload=True, verbose=False)


    Created 720 epochs, each 5 seconds long.
  Segmenting file 61/61: chb02_19_ica_cleaned.fif


/tmp/ipython-input-3105503034.py:27: RuntimeWarning: This filename (/content/drive/MyDrive/NeuroFedMeta/data/Raw_CHBMIT/processed/chb02_19_ica_cleaned.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_obj = mne.io.read_raw_fif(fif_path, preload=True, verbose=False)


    Created 720 epochs, each 5 seconds long.

Finished segmentation. Processed 61 files and created a total of 42348 segments.
Shape of combined segmented data for causal analysis: (42348, 11500)


**Reasoning**:
The segmentation process in the previous cell successfully generated `all_segmented_data`. The next step, as per the task description, is to confirm that the segmented data is saved correctly by executing the existing saving cell.



In [ ]:
import h5py
import os

# 1. Check if segmented data exists
if 'all_segmented_data' in locals() and all_segmented_data.size > 0:
    data_to_save = all_segmented_data
    print(f"Data to save found with shape: {data_to_save.shape}")
elif segmented_data_for_causal_analysis:

    data_to_save = np.vstack(segmented_data_for_causal_analysis)
    print(f"Data to save found in list form with shape: {data_to_save.shape}")
else:
    print("No segmented data available to save. Please ensure previous steps ran successfully.")
    data_to_save = None

if data_to_save is not None:
    # 2. Define filename and path
    processed_data_dir = '/content/drive/MyDrive/NeuroFedMeta/data/Raw_CHBMIT/processed_Final'
    os.makedirs(processed_data_dir, exist_ok=True)
    output_filename = os.path.join(processed_data_dir, 'segmented_eeg_data.h5')

    # 4. Use h5py.File() to create or open an HDF5 file
    with h5py.File(output_filename, 'w') as f:
        # 5. Save the NumPy array as a dataset within the HDF5 file with compression
        f.create_dataset('eeg_segments', data=data_to_save, compression="gzip", compression_opts=9)

    print(f"Successfully saved segmented EEG data to: {output_filename}")
    print(f"Saved data shape: {data_to_save.shape}")

Data to save found with shape: (42348, 11500)
